In [32]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
from sklearn.model_selection import train_test_split
import _pickle as pickle
from typing import List

In [33]:
def progbar(curr, total, full_progbar):
    """
    Progress bar used in training process.
    Reference: https://geekyisawesome.blogspot.com/2016/07/python-console-progress-bar-using-b-and.html
    """
    frac = curr/total
    filled_progbar = round(frac*full_progbar)
#     print('\r', '#'*filled_progbar + '-'*(
#         full_progbar-filled_progbar), '[{:>7.2%}]'.format(frac), end='')
    print('\r', '#'*filled_progbar + '-'*(
        full_progbar-filled_progbar), f"[{curr}/{total}, {frac:>7.2%}]", end='')

In [34]:
df = pd.read_csv("./text_emotion.csv")
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [35]:
num_dimensions = 300

batch_size = 24
lstm_units = (16, 32)
num_classes = 13
iterations = 10
max_seq_length = 20

In [36]:
if bool(input("Re-read embedding?[Y/N]>>> ").upper() == "Y"):
    with open('/home/ec2-user/data/glove.840B.300d.txt') as f:
        content = f.readlines()
    total_dict = {}
    for i in range(len(content)):
        progbar(i, len(content), 40)
        this = content[i].split()
        try:
            total_dict[this[0]] = np.array(this[1:]).astype(np.float32)
        except:
            for i in range(len(this[1:])):
                try:
                    np.array(this[1:][i]).astype(np.float32)
                except:
                    this[i+1] = '0.0'
            total_dict[this[0]] = np.array(this[1:]).astype(np.float32)

    with open('./dict.pkl', 'wb') as f:
        pickle.dump(total_dict, f)
else:
    with open('./dict.pkl', 'rb') as f:
        total_dict = pickle.load(f)

Re-read embedding?[Y/N]>>> n


In [37]:
# def s5_idx(emo) -> int:
#     if emo in ["happiness", "enthusiasm", "love", "relief", "fun"]:
#         return 0
#     elif emo in ["anger", "hate"]:
#         return 1
#     elif emo in ["sadness", "empty", "boredom", "worry"]:
#         return 2
#     elif emo in ["neutral"]:
#         return 3
#     elif emo in ["surprise"]:
#         return 4

In [65]:
# senti5 = ["happy", "aggresive", "boring", "neutral", "surprise"]
senti_lst = list(set(df["sentiment"]))
print(senti_lst)
# print(senti5)

['enthusiasm', 'relief', 'sadness', 'love', 'fun', 'hate', 'worry', 'surprise', 'happiness', 'neutral', 'anger', 'boredom', 'empty']


In [66]:
def word2vec(sentence: List[str], max_seq_length=20) -> np.ndarray:
    sentence = item.split()
    vec_lst = list()
    for word in sentence:
        if len(vec_lst) < max_seq_length and word[0] != "@":
            try:
                vec_lst.append(total_dict[word][:num_dimensions])
            except KeyError:
    #             vec_lst.append(np.zeros([num_dimensions]))
                vec_lst.append(total_dict["something"])
    while len(vec_lst) < max_seq_length:
        vec_lst.append(np.zeros([num_dimensions])
        )
    vec_ary = np.stack(vec_lst)
    return vec_ary

In [67]:
X_raw = []
y_raw = []
for i, (senti, item) in enumerate(zip(df["sentiment"], df["content"])):
    progbar(i+1, len(df), 50)
    X_raw.append(word2vec(item))
    
    # ==== Process y ====
#     y_idx = s5_idx(senti)
#     label = np.zeros([5])
    y_idx = senti_lst.index(senti)
    label = np.zeros(13)
    label[y_idx] = 1
    assert sum(label) == 1
    y_raw.append(label)
X_raw = np.array(X_raw)
y_raw = np.array(y_raw)

 -------------------------------------------------- [392/40000,   0.98%]

 #------------------------------------------------- [743/40000,   1.86%]

 #------------------------------------------------- [1109/40000,   2.77%]

 ##------------------------------------------------ [1454/40000,   3.64%]

 ##------------------------------------------------ 

...

 ##########################------------------------ [20527/40000,  51.32%]

 ##########################------------------------ [20733/40000,  51.83%]

 ##########################------------------------ [20967/40000,  52.42%]

 ##########################------------------------ [21197/40000,  52.99%]

 ################################################## [40000/40000, 100.00%]

In [68]:
print(f"X shape: {X_raw.shape}")
print(f"y shape: {y_raw.shape}")

X shape: (40000, 20, 300)
y shape: (40000, 13)


In [69]:
# for (i, truth) in zip(range(y_raw.shape[0]), df["sentiment"]):
#     idx = np.squeeze(np.where(y_raw[i, :] == 1))
#     assert senti_lst[idx] == truth

In [70]:
(X_train, X_test,
 y_train, y_test) = train_test_split(
    X_raw, y_raw,
    test_size=0.2,
    shuffle=True
)

(X_train, X_val,
 y_train, y_val) = train_test_split(
    X_train, y_train,
    test_size=0.2,
    shuffle=True
)

In [71]:
print(f"Training and testing set generated,\
\nX_train shape: {X_train.shape}\
\ny_train shape: {y_train.shape}\
\nX_test shape: {X_test.shape}\
\ny_test shape: {y_test.shape}\
\nX_validation shape: {X_val.shape}\
\ny_validation shape: {y_val.shape}")

Training and testing set generated,
X_train shape: (25600, 20, 300)
y_train shape: (25600, 13)
X_test shape: (8000, 20, 300)
y_test shape: (8000, 13)
X_validation shape: (6400, 20, 300)
y_validation shape: (6400, 13)


In [72]:
X_train_batches = X_train.reshape(50, 512, max_seq_length, num_dimensions)
y_train_batches = y_train.reshape(50, 512, num_classes)

In [73]:
cost = 0
for i in range(len(df.content[0].split())):
    try:
        cost += np.linalg.norm(total_dict[df.content[0].split()[i]][:300] - X_raw[0][i])
    except:
        cost += np.linalg.norm(total_dict["something"][:300] - X_raw[0][i])
print(cost)

129.66321728923384


In [74]:
tf.reset_default_graph()

with tf.name_scope("DATA_IO"):
    X = tf.placeholder(
        tf.float32,
        [None, max_seq_length, num_dimensions]
    )
    y = tf.placeholder(
        tf.float32,
        [None, num_classes]
    )
cell_types = [
    tf.nn.rnn_cell.LSTMCell,
    tf.nn.rnn_cell.GRUCell,
]
with tf.name_scope("RNN"):
    cell = tf.nn.rnn_cell.MultiRNNCell(
        [cell_type(
            num_units=units,
            name=f"RNN_LAYER_{i}")
            for i, (cell_type, units)
            in enumerate(zip(cell_types, lstm_units))
         ])
#     cell = tf.contrib.rnn.LSTMCell(
#         num_units=lstm_units
#     )

    lstm_cell = tf.contrib.rnn.DropoutWrapper(
        cell=cell,
        output_keep_prob=0.75
    )
    outputs, state = tf.nn.dynamic_rnn(
        lstm_cell, 
        X, 
        dtype=tf.float32
    )

with tf.name_scope("OUTPUT"):
    weight = tf.Variable(
        tf.truncated_normal(
            [lstm_units[-1], num_classes]
        )
    )
    
    bias = tf.Variable(
        tf.random_normal(shape=[num_classes])
    )

# Option i)
#     value = tf.transpose(value, [1, 0, 2])
#     last = tf.gather(value, int(value.get_shape()[0]) - 1)
# Option ii)
    last = outputs[:, -1, :]
    pred = tf.matmul(last, weight) + bias
    pred_idx = tf.argmax(pred, axis=1)

In [75]:
with tf.name_scope("METRICS"):
    correct_pred = tf.equal(
        tf.argmax(pred, axis=1),
        tf.argmax(y, axis=1)
    )

    accuracy = tf.reduce_mean(
        tf.cast(correct_pred, tf.float32)
    )

with tf.name_scope("LOSSES"):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=pred,
            labels=y
        )
    )
    optimizer = tf.train.AdamOptimizer().minimize(loss)

tf.summary.scalar("Loss", loss)
tf.summary.scalar("Accuracy", accuracy)

<tf.Tensor 'Accuracy:0' shape=() dtype=string>

In [130]:
sess = tf.Session()
saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())

merged = tf.summary.merge_all()
logdir = "./tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

for e in range(10):
    for X_batch, y_batch in zip(X_train_batches, y_train_batches):
        sess.run(
            optimizer,
            feed_dict={
                X: X_batch,
                y: y_batch
            }
        )

    if e % 5 == 0:
        summary = sess.run(
            merged,
            feed_dict={
                X: X_val,
                y: y_val
            }
        )
    if e % 1 == 0:
        train_acc = accuracy.eval(feed_dict={X: X_train, y: y_train}, session=sess)
        val_acc = accuracy.eval(feed_dict={X: X_val, y: y_val}, session=sess)
        print(f"Epochs[{e}]: train accuracy ={train_acc}, val accuracy={val_acc}")
    writer.add_summary(summary, e)
    writer.close()
f = lambda src: pred_idx.eval(feed_dict={X: src}, session=sess)
train_pred = f(X_train)
test_pred = f(X_test)
val_pred = f(X_val)

Epochs[0]: train accuracy =0.20121093094348907, val accuracy=0.19234375655651093
Epochs[1]: train accuracy =0.2679687440395355, val accuracy=0.2628124952316284
Epochs[2]: train accuracy =0.28339844942092896, val accuracy=0.28046876192092896
Epochs[3]: train accuracy =0.2982812523841858, val accuracy=0.2903124988079071
Epochs[4]: train accuracy =0.30324217677116394, val accuracy=0.28999999165534973
Epochs[5]: train accuracy =0.3101562559604645, val accuracy=0.29906249046325684
Epochs[6]: train accuracy =0.3182031214237213, val accuracy=0.3032812476158142
Epochs[7]: train accuracy =0.32691407203674316, val accuracy=0.3062500059604645
Epochs[8]: train accuracy =0.3330078125, val accuracy=0.3010937571525574
Epochs[9]: train accuracy =0.3381640613079071, val accuracy=0.3123437464237213


In [128]:
s = "It is such a nice day "

In [129]:
idx = pred_idx.eval(feed_dict={X: word2vec(s).reshape(1, max_seq_length, num_dimensions)}, session=sess)
print(senti_lst[int(idx)])

sadness


In [81]:
np.mean(np.argmax(y_test, axis=1) == test_pred)

0.318625

In [ ]:
n = 28888
print(df["content"][n])
print(df["sentiment"][n])

In [ ]:
sum(df["sentiment"] == "happiness") / len(df)